In [136]:
import keras
from keras.datasets import cifar10

In [137]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # x가 rgb를 포함한 data, y가 label data

In [138]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [139]:
import matplotlib.pyplot as plt
import numpy as np

In [140]:
from keras.utils import to_categorical

In [141]:
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [142]:
print(x_train.shape) # 이미지수 , 이미지 크기, rgb
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)
print(x_test.shape)
print(y_test.shape)

(45000, 32, 32, 3)
(45000, 1)
(5000, 32, 32, 3)
(5000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [143]:
# 데이터 정규화
mean = np.mean(x_train, axis=(0,1,2,3))
std = np.std(x_train, axis=(0,1,2,3))

In [144]:
print(mean,std)

120.735214 64.12227


In [145]:
x_train = (x_train - mean) / (std + 1e-7)
x_valid =(x_valid - mean) / (std + 1e-7)
x_test = (x_test - mean) / (std + 1e-7)

In [146]:
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes) #to_categorical ==> one-hot encoding
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

In [164]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras import regularizers

In [165]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.attention.multi_head_attention import regularization

In [166]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D, AveragePooling2D
from keras.layers import Dense, Flatten, Dropout, BatchNormalization

------------------------------model1 ----------------------그대로적용 cifar10

In [90]:
model = Sequential()

model.add(Conv2D(filters=6, kernel_size=5, padding='same',  activation='tanh', input_shape =(32,32,3)))
model.add(AveragePooling2D(pool_size=2))
model.add(Conv2D(filters=16, kernel_size=5, activation='tanh'))
model.add(AveragePooling2D(pool_size=2))



model.add(Flatten())

model.add(Dense(120,activation="tanh"))
model.add(Dense(84,activation="tanh"))
model.add(Dense(10,activation="softmax"))


In [91]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 32, 32, 6)         456       
                                                                 
 average_pooling2d_18 (Avera  (None, 16, 16, 6)        0         
 gePooling2D)                                                    
                                                                 
 conv2d_22 (Conv2D)          (None, 12, 12, 16)        2416      
                                                                 
 average_pooling2d_19 (Avera  (None, 6, 6, 16)         0         
 gePooling2D)                                                    
                                                                 
 flatten_8 (Flatten)         (None, 576)               0         
                                                                 
 dense_23 (Dense)            (None, 120)             

In [92]:
def lr_schedule(epoch) :
  if epoch <= 2:
    lr = 5e-4
  elif epoch > 2 and epoch <= 5 :
    lr = 2e-4
  elif epoch > 5 and epoch <= 9 :
    lr = 5e-5
  else :
    lr = 1e-5
  return lr

In [93]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

lr_scheduler = LearningRateScheduler(lr_schedule)
checkpoint = ModelCheckpoint(filepath='path_to_save_file/file.hdf5',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint, lr_scheduler]


model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [94]:
hist = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), callbacks = callbacks, verbose=2, shuffle = True)

Epoch 1/20

Epoch 1: val_accuracy improved from -inf to 0.25800, saving model to path_to_save_file/file.hdf5
1407/1407 - 6s - loss: 2.2250 - accuracy: 0.1842 - val_loss: 2.1054 - val_accuracy: 0.2580 - lr: 5.0000e-04 - 6s/epoch - 5ms/step
Epoch 2/20

Epoch 2: val_accuracy improved from 0.25800 to 0.30480, saving model to path_to_save_file/file.hdf5
1407/1407 - 5s - loss: 2.0369 - accuracy: 0.2838 - val_loss: 1.9735 - val_accuracy: 0.3048 - lr: 5.0000e-04 - 5s/epoch - 4ms/step
Epoch 3/20

Epoch 3: val_accuracy improved from 0.30480 to 0.32300, saving model to path_to_save_file/file.hdf5
1407/1407 - 6s - loss: 1.9510 - accuracy: 0.3145 - val_loss: 1.9233 - val_accuracy: 0.3230 - lr: 5.0000e-04 - 6s/epoch - 4ms/step
Epoch 4/20

Epoch 4: val_accuracy improved from 0.32300 to 0.33130, saving model to path_to_save_file/file.hdf5
1407/1407 - 5s - loss: 1.9223 - accuracy: 0.3245 - val_loss: 1.9106 - val_accuracy: 0.3313 - lr: 2.0000e-04 - 5s/epoch - 4ms/step
Epoch 5/20

Epoch 5: val_accuracy i

In [95]:
scores=model.evaluate(x_test,y_test, batch_size=128, verbose=1)

79/79 [==============================] - 0s 4ms/step - loss: 1.8773 - accuracy: 0.3459


In [96]:
scores[0], scores[1]

(1.8772826194763184, 0.3458999991416931)

-------------------------model2-------------------mnist data

In [109]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)   # Tensorflow의 버전을 출력

mnist = keras.datasets.mnist

# MNIST 데이터를 로드. 다운로드하지 않았다면 다운로드까지 자동으로 진행됩니다. 
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

print(len(x_train))  # x_train 배열의 크기를 출력
print(len(y_train))
print(len(x_test))
print(len(y_test))

2.9.2
60000
60000
10000
10000


In [117]:
model2 = Sequential()

model2.add(Conv2D(filters=6, kernel_size=5, padding='same',  activation='tanh', input_shape =(28,28,1)))
model2.add(AveragePooling2D(pool_size=2))
model2.add(Conv2D(filters=16, kernel_size=5, activation='tanh'))
model2.add(AveragePooling2D(pool_size=2))



model2.add(Flatten())

model2.add(Dense(120,activation="tanh"))
model2.add(Dense(84,activation="tanh"))
model2.add(Dense(10,activation="softmax"))


In [118]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

lr_scheduler = LearningRateScheduler(lr_schedule)
checkpoint = ModelCheckpoint(filepath='path_to_save_file/file2.hdf5',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint, lr_scheduler]


model2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [119]:
hist2 = model2.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), callbacks = callbacks, verbose=2, shuffle = True)

Epoch 1/20


ValueError: ignored

In [116]:
scores=model2.evaluate(x_test,y_test, batch_size=128, verbose=1)

ValueError: ignored

In [75]:
scores[0], scores[1]

(1.8400697708129883, 0.3603000044822693)

-------------------------model3-------------------relu

In [97]:
 model3 = Sequential()

 model3.add(Conv2D(filters=6, kernel_size=5, padding='same',  activation='relu', input_shape =(32,32,3)))
 model3.add(AveragePooling2D(pool_size=2))
 model3.add(Conv2D(filters=16, kernel_size=5, activation='relu'))
 model3.add(AveragePooling2D(pool_size=2))



 model3.add(Flatten())

 model3.add(Dense(120,activation="relu"))
 model3.add(Dense(84,activation="relu"))
 model3.add(Dense(10,activation="softmax"))


In [98]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

lr_scheduler = LearningRateScheduler(lr_schedule)
checkpoint = ModelCheckpoint(filepath='path_to_save_file/file3.hdf5',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint, lr_scheduler]


model3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [99]:
hist3 = model3.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), callbacks = callbacks, verbose=2, shuffle = True)

Epoch 1/20

Epoch 1: val_accuracy improved from -inf to 0.18050, saving model to path_to_save_file/file3.hdf5
1407/1407 - 7s - loss: 2.2724 - accuracy: 0.1412 - val_loss: 2.2295 - val_accuracy: 0.1805 - lr: 5.0000e-04 - 7s/epoch - 5ms/step
Epoch 2/20

Epoch 2: val_accuracy improved from 0.18050 to 0.24070, saving model to path_to_save_file/file3.hdf5
1407/1407 - 6s - loss: 2.1840 - accuracy: 0.2071 - val_loss: 2.1268 - val_accuracy: 0.2407 - lr: 5.0000e-04 - 6s/epoch - 4ms/step
Epoch 3/20

Epoch 3: val_accuracy improved from 0.24070 to 0.28070, saving model to path_to_save_file/file3.hdf5
1407/1407 - 6s - loss: 2.0812 - accuracy: 0.2609 - val_loss: 2.0266 - val_accuracy: 0.2807 - lr: 5.0000e-04 - 6s/epoch - 4ms/step
Epoch 4/20

Epoch 4: val_accuracy improved from 0.28070 to 0.29560, saving model to path_to_save_file/file3.hdf5
1407/1407 - 5s - loss: 2.0161 - accuracy: 0.2842 - val_loss: 1.9934 - val_accuracy: 0.2956 - lr: 2.0000e-04 - 5s/epoch - 4ms/step
Epoch 5/20

Epoch 5: val_accura

In [100]:
scores=model3.evaluate(x_test,y_test, batch_size=128, verbose=1)

79/79 [==============================] - 1s 9ms/step - loss: 1.9001 - accuracy: 0.3277


In [101]:
scores[0], scores[1]

(1.900071620941162, 0.3276999890804291)

------------------

## AlexNet


In [170]:
from keras.regularizers import l2

In [183]:


alexnet = Sequential()

alexnet.add(Conv2D(filters=64, kernel_size=7, padding='valid', activation='relu', input_shape =(32,32,3)))

alexnet.add(BatchNormalization())
alexnet.add(Conv2D(filters=192, kernel_size=5, strides = 1, padding="same", activation='relu', kernel_regularizer=l2(0.0005)))
alexnet.add(Dropout(0.3))
alexnet.add(MaxPooling2D(pool_size=3, strides=2))
alexnet.add(BatchNormalization())
alexnet.add(Conv2D(filters=256, kernel_size=3, strides = 1, padding='same', activation='relu', kernel_regularizer=l2(0.0005)))
alexnet.add(Dropout(0.4))
alexnet.add(Conv2D(filters=256, kernel_size=3, strides = 1, padding='same', activation='relu', kernel_regularizer=l2(0.0005)))
alexnet.add(Conv2D(filters=128, kernel_size=3, strides = 1, padding='same', activation='relu', kernel_regularizer=l2(0.0005)))
alexnet.add(MaxPooling2D(pool_size=3, strides=2))

alexnet.add(Flatten())


alexnet.add(Dense(3200,activation="relu"))
alexnet.add(Dropout(0.5))
alexnet.add(Dense(2048,activation="relu"))
alexnet.add(Dropout(0.5))
alexnet.add(Dense(10,activation="softmax"))


In [184]:
alexnet.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_88 (Conv2D)          (None, 26, 26, 64)        9472      
                                                                 
 batch_normalization_17 (Bat  (None, 26, 26, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_89 (Conv2D)          (None, 26, 26, 192)       307392    
                                                                 
 dropout_8 (Dropout)         (None, 26, 26, 192)       0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 12, 12, 192)      0         
 g2D)                                                            
                                                                 
 batch_normalization_18 (Bat  (None, 12, 12, 192)    

In [185]:
256 *36

9216

In [186]:
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD
import numpy as np

In [187]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1))

optimizer =SGD(learning_rate=0.001, momentum=0.9)

alexnet.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

alexnet.fit(x_train,y_train, 
            batch_size=128,
            epochs=90,
            validation_data=(x_valid, y_valid),
            verbose=2,
            callbacks=[reduce_lr])

Epoch 1/90
352/352 - 18s - loss: 2.2516 - accuracy: 0.2956 - val_loss: 2.5630 - val_accuracy: 0.2248 - lr: 0.0010 - 18s/epoch - 52ms/step
Epoch 2/90
352/352 - 17s - loss: 1.8784 - accuracy: 0.4406 - val_loss: 2.1536 - val_accuracy: 0.3404 - lr: 0.0010 - 17s/epoch - 48ms/step
Epoch 3/90
352/352 - 17s - loss: 1.7212 - accuracy: 0.5047 - val_loss: 1.8765 - val_accuracy: 0.4382 - lr: 0.0010 - 17s/epoch - 47ms/step
Epoch 4/90
352/352 - 17s - loss: 1.5954 - accuracy: 0.5534 - val_loss: 1.7803 - val_accuracy: 0.4880 - lr: 0.0010 - 17s/epoch - 47ms/step
Epoch 5/90
352/352 - 17s - loss: 1.4972 - accuracy: 0.5921 - val_loss: 1.8112 - val_accuracy: 0.4606 - lr: 0.0010 - 17s/epoch - 48ms/step
Epoch 6/90
352/352 - 17s - loss: 1.4236 - accuracy: 0.6182 - val_loss: 1.7951 - val_accuracy: 0.4802 - lr: 0.0010 - 17s/epoch - 48ms/step
Epoch 7/90
352/352 - 17s - loss: 1.3681 - accuracy: 0.6376 - val_loss: 1.7142 - val_accuracy: 0.5086 - lr: 0.0010 - 17s/epoch - 48ms/step
Epoch 8/90
352/352 - 17s - loss: 1